In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'20',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '114897c7-fc98-4d4f-b667-bbd6c56cd6b4',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
type(data)


In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


df = pd.json_normalize(data['data'])
df['timestamp'] = pd.to_datetime('now')
df

In [ ]:
def api_runner():
    global df
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
         'start':'1',
         'limit':'20',
         'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': '114897c7-fc98-4d4f-b667-bbd6c56cd6b4',
    }

    session = Session()
    session.headers.update(headers)

    try:
      response = session.get(url, params=parameters)
      data = json.loads(response.text)
      print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
     print(e)
    
    
    df2 = pd.json_normalize(data['data'])
    df2['timestamp'] = pd.to_datetime('now')
    df = df.append(df2)
    
    
    if not os.path.isfile(r'/Users/neelrajwat/API.csv'):
        df.to_csv('/Users/neelrajwat/API.csv', header = 'column_names')
    else:
        df.to_csv('/Users/neelrajwat/API.csv', mode = 'a', header = False)

In [ ]:
import os
from time import time
from time import sleep

for i in range(222):
    api_runner()
    print("API run completed.")
    sleep(60)
exit()

In [ ]:
df4 = pd.read_csv(r'/Users/neelrajwat/API.csv')
df4

In [ ]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
df4

In [ ]:
df5 = df4.groupby('name', sort = False)[['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d']].mean()
df5

In [ ]:
df6 = df5.stack()
df6


In [ ]:
type(df6)

In [ ]:
df7 = df6.to_frame(name='values')
df7

In [ ]:
df7.count()

In [ ]:
index = pd.Index(range(120))

df8 = df7.reset_index()
df8

In [ ]:
df9 = df8.rename(columns = {'level_1':'duration'})
df9

In [ ]:
df9['duration'] = df9['duration'].replace(['quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d','quote.USD.percent_change_60d','quote.USD.percent_change_90d'],['1H','24H','7D','30D','60D','90D'])
df9

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.scatterplot(x='duration',y='values', hue = 'name', data = df9)
plt.legend(title = 'cryptocurrencies', title_fontsize = 'x-large', shadow = True, loc='upper left', bbox_to_anchor=(1.1, 1.1))
plt.show()

In [ ]:
df10 = df[['name', 'quote.USD.price','timestamp']]
df10 = df10.query("name == 'Ethereum'")
df10

In [ ]:
df10['timestamp'] = df10['timestamp'].replace(['2023-04-27 16:18:01.329977'], ['16:18'])
df10

In [ ]:
sns.set_theme(style='darkgrid')

line_plot = sns.lineplot(x='timestamp',y='quote.USD.price', data = df10)
plt.ylabel("Price in USD")
line_plot.set_xticklabels(line_plot.get_xticklabels(), rotation=45)
